<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px">

# Cubeviz with NIRSpec data

**Author: Jesse Averbukh | Software Engineer at STScI**<br>
**Last update: May 27, 2025**

## Introduction
This tutorial will cover the following features:
- Moment maps
- Spectrum when hovering over spaxel
- Spectral extraction
- Unit conversion

The data used in this tutorial is provided courtesy of MAST and the analysis is based on the paper "GOALS-JWST: Gas Dynamics and Excitation in NGC 7469 Revealed by NIRSpec" ( https://iopscience.iop.org/article/10.3847/1538-4357/ad2a50/pdf ) by Marina Bianchin et al. (2024)

## Imports
We will start with our imports:

In [1]:
# This is where we import the cubeviz configuration
from jdaviz import Cubeviz

import warnings

# For creating spectral subsets
from specutils import SpectralRegion
import astropy.units as u

# For creating spatial subsets
from regions import PixCoord, CirclePixelRegion, RectanglePixelRegion

In [2]:
# Create an instance of cubeviz
cubeviz = Cubeviz()

In [3]:
# cubeviz.show('sidecar:right')
cubeviz.show()

Application(config='cubeviz', docs_link='https://jdaviz.readthedocs.io/en/v4.2.3/cubeviz/index.html', events=[…

In [4]:
# Use MAST URI
uri = 'mast:JWST/product/jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits'

# pre-downloaded file
# filepath = '../data/jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits'

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(uri, cache=True)

 [Done]


## Subsets

In [5]:
# Access the subset plugin
subset_plugin = cubeviz.plugins['Subset Tools']

In [6]:
# Retrieve the spectral units
spectral_axis_unit = u.Unit(cubeviz.plugins['Unit Conversion'].spectral_unit.selected)

In [7]:
# Highlight the Paschen-alpha line with a spectral subset
pa_spec_region = SpectralRegion(1.9033 * spectral_axis_unit,
                                1.9085 * spectral_axis_unit)
subset_plugin.import_region(pa_spec_region)

## Moment maps

Now that we have the  Paschen-alpha line highlighted, we can create a moment map to get a better sense of what is happening in the galaxy. We will start with a moment 0 map, which will show us the concentration of the particular element in the galaxy.

In [8]:
# Create a moment map of 0 for the PA emission line
mm = cubeviz.plugins['Moment Maps']
mm.open_in_tray()
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 0
mm.add_results.label = 'moment-0-pa'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], shape=(59, 55), unit='MJy um / sr')

The paper makes some observations about the increased amounts of Paschen-alpha corresponding to star-forming regions in the north, west, and southern areas of the outer ring. There also appears to be a slight trail of this element going from the nucleus to the ring just south east of the nucleus.

We will now create moment maps with a moment of 1 to see the flux-weighted centroid of the Paschen-alpha line. This will show us the velocity field in the galaxy at that wavelength.

In [9]:
# Create a moment map of 1 for the Paschen-alpha emission line
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 1
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-1-pa'
mm.add_results.viewer = 'flux-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], shape=(59, 55), unit='km / s')

In [10]:
# Create a moment map of 2 for the Paschen-alpha emission line
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 2
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-2-pa'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

CCDData([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]], shape=(59, 55), unit='km / s')

## Plot options

In [11]:
po = cubeviz.plugins['Plot Options']
po.open_in_tray()
po.viewer = 'flux-viewer'
po.layer = 'moment-1-pa'
po.image_colormap.value = 'rainbow'

In [12]:
po.viewer = 'uncert-viewer'
po.layer = 'moment-2-pa'
po.stretch_preset.value = 90
po.image_colormap.value = 'rainbow'

Positive values in the flux and uncertainty viewers correspond to redshifted areas and negative values correspond to blueshifted areas. This means the galaxy is rotating in a counter-clockwise direction. We can find the minimum and maximum values to get an estimate of the speed of rotation.

For the moment 2 map, we can see the outer ring is relatively calm with values close to the minimum threshold of the detector, while the nucleus is very turbulent. This is another sign of a nuclear radio jet coming from the AGN in the nucleus.

## Spectral extraction

In [13]:
# This sets the combination mode to be new, which means the 
# next region added will be a new subset
subset_plugin.combination_mode = 'new'

# Create spatial subset around large Paschen-alpha area
# We can also set the combination mode in the method so this will be a new subset
subset_plugin.import_region(RectanglePixelRegion(center=PixCoord(x=27.5, y=45.5), width=5, height=9))

In [14]:
# Create spatial subset around the nucleus
subset_plugin.import_region(
    CirclePixelRegion(center=PixCoord(x=23.5, y=29.2), radius=5),
    combination_mode='new')

We can see that Subset 2 is highlighting one of the strongest regions of star formation. Subset 3 is highlighting the strong line flux map in the nucleus. We can use the Spectral Extraction plugin to see how the nucleus looks with a conical aperture.

In [15]:
# We use a conical aperture for the nucleus because it is a PSF (point spread function)
se = cubeviz.plugins['Spectral Extraction']
se.open_in_tray()
se.aperture = 'Subset 3'
se.wavelength_dependent = True
se.aperture_method = 'Exact'
se.function = 'Sum'
se.extract()

<Spectrum1D(flux=[2.203460267814017e-08 ... 0.0] MJy (shape=(3814,), mean=0.00000 MJy); spectral_axis=<SpectralAxis 
   (observer: <ICRS Coordinate: (ra, dec, distance) in (deg, deg, m)
                  (297.97652647, -21.33633333, 1.50842438e+11)
               (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
                  (0., 0., 0.)>
    observer to target:
      radial_velocity=0.0 km / s
      redshift=0.0)
  [1.66019797 1.66059397 1.66098997 ... 3.16935392 3.16974992 3.17014592] um> (length=3814); uncertainty=StdDevUncertainty)>